In [1]:
import numpy as np
from datetime import datetime

# Message processing

In [10]:
#Transform the text message to binary
def toBinary(message):
     return ''.join(format(ord(char), '08b') for char in message)

In [3]:
def pad_and_create_Blocks(bstring):
    l = len(bstring)
    temp = bstring
    blocks = []
    #Dividing the binary represenation of a message into blocks of 512 bits
    while len(temp)>0:
        block = temp[:512]
        if len(block) < 512:     # Padding blocks of less than 512 bits
            len_final_block = len(block)
            bin_len = '{:08b}'.format(len_final_block)
            block = temp + '1' + '0'*(512-len_final_block - len(bin_len) -1) + bin_len
            
        blocks.append(block)
        temp = temp[512:]
    return blocks


# SHA1 algorithm

In [4]:
def left_rot(data, n):
    return ((data << n) | (data >> (32 - n))) & 0xffffffff  #returns data left-shifted by n

In [21]:
def SHA1(data):
    data = toBinary(data)
    blocks = pad_and_create_Blocks(data)
    
    h0 = 0x67452301              #SHA1 chaining variables
    h1 = 0xEFCDAB89
    h2 = 0x98BADCFE
    h3 = 0x10325476
    h4 = 0xC3D2E1F0

    K = [
    0x5A827999, # ( 0 <= i <= 19)     #SHA1 iteration constants
    0x6ED9EBA1, # (20 <= i <= 39)
    0x8F1BBCDC, # (40 <= i <= 59)
    0xCA62C1D6  # (60 <= i <= 79)
    ]
    
    for block in blocks:
        a = h0
        b = h1
        c = h2
        d = h3
        e = h4

        #divide each 512-bit block into 16 blocks of 32 bits to create w-values
        sub_blocks = []
        while len(block)>0:                           
            sub_blocks.append(block[:32])
            block = block[32:]
        
        w = [0]*80
        for i in range(0, 16):             # First 16 values of w is the content of sub-blocks
            w[i] = int(sub_blocks[i], 2)
        for i in range(16, 80):
            w[i] = left_rot(w[i-3] ^ w[i-8] ^ w[i-14] ^ w[i-16], 1)  # formula for calculating the rest of w, 16-80
        
        for i in range(0, 80):     # the 80 iterations of SHA1
            if 0 <= i <= 19:
                p = (b & c) | ((~b) & d)
                k = K[0]
            elif 20 <= i <= 39:
                p = b ^ c ^ d
                k = K[1]
            elif 40 <= i <= 59:
                p = (b & c) | (b & d) | (c & d) 
                k = K[2]
            elif 60 <= i <= 79:
                p = b ^ c ^ d
                k = K[3]

            temp = left_rot(a, 5) + p + e + k + w[i] & 0xffffffff
            e = d
            d = c
            c = left_rot(b, 30)
            b = a
            a = temp
        #updating h0-h4, & 0xffffffff is used as a mask to do 32-bit integer math
        h0 = (h0 + a) & 0xffffffff
        h1 = (h1 + b) & 0xffffffff
        h2 = (h2 + c) & 0xffffffff
        h3 = (h3 + d) & 0xffffffff
        h4 = (h4 + e) & 0xffffffff
        h = [h0, h1, h2, h3, h4]
    return "".join(hex(i)[2:] for i in h)

# Merkle Tree

In [22]:
def merkle(trans): #Takes as input a set of transactions
    h_trans = []
    for t in trans:
        h_trans.append(SHA1(t))   # Transform each transaction and store

    while len(h_trans) > 1:
        temp = []
        if not(len(h_trans) % 2 == 0):    # If transactions are not paired, add a copy of last transaction
            h_trans.append(h_trans[-1])
        
        for i in range(0, len(h_trans), 2): # Hash each pair and store in "temp"
            trans1 = h_trans[i]
            trans2 = h_trans[i+1]

            temp.append(SHA1(trans1 + trans2))
         
        h_trans = temp #finally, overwrite h_trans with the new hashed pairs

    return h_trans

# Block-chaining

In [7]:
def new_block(block_nr, prev_hash, current_hash, transactions):     #Dictionary to hold block information
    block = {
    "Header": "Block" + str(block_nr),
    "timestamp": datetime.now(),
    "PrevHash": prev_hash,
    "Tx_Root": current_hash,
    "transactions": transactions
    }
    return block
def print_block(block):                                            #Print function for a block
    print("header:", block["Header"])
    print("timestamp:", block["timestamp"])
    print("PrevHash:", block["PrevHash"])
    print("Tx_Root:", block["Tx_Root"])
    print("transactions:", block["transactions"])

In [18]:
prompt = input("Press enter to start blockchain: ")
if prompt == "":
    start = True

counter = 0
blockchain = []
while(start):
    counter += 1

    transactions = []
    # Collect transactions and store them in transactions list
    while(True):
        trans = input("Add a transaction or press enter to end block: ")
        if trans == '':
            break
        else:
            transactions.append(trans)
    hash_transactions = merkle(transactions)
    if counter == 1: 
        prev_hash = 'no previous hash'  # The first block in the chain, therefore no previous block
    else:
        prev_hash = blockchain[counter-2]["Tx_Root"]
    block = new_block(counter, prev_hash, hash_transactions, transactions)
    
    print("Your new block is: ")
    print_block(block)
    blockchain.append(block)
    print("")
    print("Current blockchain:")
    
    for block in blockchain:
        print_block(block)
    
    prompt = eval(input("Enter '1' for new block or '2' to end chain: "))
    if prompt == 1:
        continue
    elif prompt == 2:
        start = False
        print('Blockchain session ended.')
    

    
    

Press enter to start blockchain: 
Add a transaction or press enter to end block: Transaction 1
Add a transaction or press enter to end block: Transaction 2
Add a transaction or press enter to end block: 
Your new block is: 
header: Block1
timestamp: 2019-10-30 21:43:07.883964
PrevHash: no previous hash
Tx_Root: ['2030dba9d388a634b71ccc12477802a1a9b72d3a']
transactions: ['Transaction 1', 'Transaction 2']

Current blockchain:
header: Block1
timestamp: 2019-10-30 21:43:07.883964
PrevHash: no previous hash
Tx_Root: ['2030dba9d388a634b71ccc12477802a1a9b72d3a']
transactions: ['Transaction 1', 'Transaction 2']
Enter '1' for new block or '2' to end chain: 1
Add a transaction or press enter to end block: Transaction 3
Add a transaction or press enter to end block: Transaction 4
Add a transaction or press enter to end block: 
Your new block is: 
header: Block2
timestamp: 2019-10-30 21:54:33.013794
PrevHash: ['2030dba9d388a634b71ccc12477802a1a9b72d3a']
Tx_Root: ['96659ef7706e34a5bd0a2d64b1709d60d